<a href="https://colab.research.google.com/github/stazam/M7DataSp---Advanced-data-science-techniques/blob/main/HocekyBetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hockey_datas.csv to hockey_datas.csv


In [3]:
df = pd.read_csv("hockey_datas.csv")

In [4]:
df.shape

(1396, 23)

In [5]:
df.head()

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away
0,0,83430,1,11/09/2015,"HK Poprad, s.r.o.",HKM a.s. Zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0,0.0
1,1,83431,2,11/09/2015,"HK DUKLA Trenčín, a.s.",MHC MOUNTFIELD Martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0,0.0
2,2,83432,3,11/09/2015,"HK 36 Skalica, s.r.o.","ŠHK 37 Piešťany, s.r.o.",HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0,0.0
3,3,83433,4,11/09/2015,HC Košice s.r.o.,MsHK DOXXbet Žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0,0.0
4,4,83434,5,10/09/2015,HC ‘05 iClinic BANSKÁ BYSTRICA,"HK Nitra, a.s.",ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0,0.0


1. pridat stplpec result - HOTOVO
2. zmeit stlpec date na datetime premennu
3. upravit vsetky hodnoty v stlpcoch
4. pridat sltpce s  premennymi ktore mal ten typek
6. pozot na chybajuce pozorovania pri faceoff, places a skontorlovat aj dalsie a upravit a nahradit medianom alebo takym niecim
7. zistit unikatne hodnoty a podla toho to upravit

conceded = pocet inkasovanych golov

H2H v specifickej sezone a sucasne aj za celkove predchadzajuce obdobie

vitazstva celkovo za vsetky sezony, vitazstva celkovo za danu sezonu proti danemu superovi

In [6]:
def name_parser(x):

  x = x.lower()
  clubs = ['Poprad', 'Trenčín', 'Skalica', 'Košice', 'BANSKÁ BYSTRICA', 'Zvolen', 'Piešťany', 'Martin', 'Nitra', 'Žilina', 
           'Liptovský Mikuláš', 'Nové Zámky', 'Detva', 'Újbuda', 'Miskolc', 'Michalovce', 'Bratislava']

  clubs = [x.lower() for x in clubs]
  for club in clubs:
    if club in x:
      return club
  
  return 

df['home'] = df['home'].apply(name_parser)
df['away'] = df['away'].apply(name_parser)

In [7]:
#mozno by som este mohol pridat to, ze ked nie je domaci tym na domacom lade, tak ako sa to premietne do vysledku -- ale to sa stava v minime pripadov
import datetime

df['date'] = pd.to_datetime(df['date'],format = "%d/%m/%Y")


In [8]:
df.loc[df['score_home'] > df['score_away'], 'result'] = 'home'
df.loc[df['score_home'] < df['score_away'], 'result'] = 'away'
df.loc[df['win_periods_home'] + df['draw_periods'] + df['win_periods_away'] >= 4, 'result'] = 'draw'

In [9]:
def season(x):

  if x >= 83430 and x <= 83699:
    return "2015/2016"
  elif x >= 87717 and x <= 87986:
    return "2016/2017"
  elif x >= 96815 and x <= 97084:
    return "2017/2018"
  elif x >= 101458 and x <= 101787:
    return "2018/2019"
  else:
    return "2019/2020"

df['season'] = df['id'].apply(season)

In [10]:
df

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away,result,season
0,0,83430,1,2015-09-11,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,home,2015/2016
1,1,83431,2,2015-09-11,trenčín,martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0000,0.0000,home,2015/2016
2,2,83432,3,2015-09-11,skalica,piešťany,HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0000,0.0000,away,2015/2016
3,3,83433,4,2015-09-11,košice,žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0000,0.0000,draw,2015/2016
4,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,home,2015/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1391,110103,358,2020-02-21,miskolc,újbuda,"Ice rink Miskolc, HUN",1385,1,0,2,1,3,0.0222,0.1000,0.9259,0.9778,0.0000,0.2500,0.7500,1.0000,0.5574,0.4426,away,2019/2020
1392,1392,110105,360,2020-02-21,košice,nové zámky,STEEL Arena Košice,7067,2,1,0,7,2,0.1892,0.0952,0.9048,0.8108,0.1429,0.2500,0.7500,0.8571,0.5536,0.4464,home,2019/2020
1393,1393,110106,361,2020-02-21,trenčín,poprad,"ZŠ Pavla Demitru, Trenčín",3120,1,0,3,3,4,0.0000,0.1212,0.8788,0.9143,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2019/2020
1394,1394,110107,362,2020-02-21,bratislava,liptovský mikuláš,Arena Ondreja Nepelu,4697,3,0,0,11,0,0.2500,0.0000,1.0000,0.7500,0.5000,0.0000,1.0000,0.5000,0.5893,0.4107,home,2019/2020


In [11]:
df_home = df.copy()  
df_away = df.copy()

In [12]:
df_home.columns = df_home.columns.str.replace("home", "team")
df_home.columns = df_home.columns.str.replace("away", "opponent")
df_home = df_home.replace('home', 'win')
df_home = df_home.replace('away', 'lost')
df_home['h_a'] = 'home'

In [13]:
df_home

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a
0,0,83430,1,2015-09-11,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,win,2015/2016,home
1,1,83431,2,2015-09-11,trenčín,martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0000,0.0000,win,2015/2016,home
2,2,83432,3,2015-09-11,skalica,piešťany,HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0000,0.0000,lost,2015/2016,home
3,3,83433,4,2015-09-11,košice,žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0000,0.0000,draw,2015/2016,home
4,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1391,110103,358,2020-02-21,miskolc,újbuda,"Ice rink Miskolc, HUN",1385,1,0,2,1,3,0.0222,0.1000,0.9259,0.9778,0.0000,0.2500,0.7500,1.0000,0.5574,0.4426,lost,2019/2020,home
1392,1392,110105,360,2020-02-21,košice,nové zámky,STEEL Arena Košice,7067,2,1,0,7,2,0.1892,0.0952,0.9048,0.8108,0.1429,0.2500,0.7500,0.8571,0.5536,0.4464,win,2019/2020,home
1393,1393,110106,361,2020-02-21,trenčín,poprad,"ZŠ Pavla Demitru, Trenčín",3120,1,0,3,3,4,0.0000,0.1212,0.8788,0.9143,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2019/2020,home
1394,1394,110107,362,2020-02-21,bratislava,liptovský mikuláš,Arena Ondreja Nepelu,4697,3,0,0,11,0,0.2500,0.0000,1.0000,0.7500,0.5000,0.0000,1.0000,0.5000,0.5893,0.4107,win,2019/2020,home


In [14]:
df_away.columns = df_away.columns.str.replace("away", "team")
df_away.columns = df_away.columns.str.replace("home", "opponent")
df_away = df_away.replace('away', 'win')
df_away = df_away.replace('home', 'lost')
df_away['h_a'] = 'away'

In [15]:
df_home.dtypes

Unnamed: 0                       int64
id                               int64
game_number                      int64
date                    datetime64[ns]
team                            object
opponent                        object
place                           object
viewers                          int64
win_periods_team                 int64
draw_periods                     int64
win_periods_opponent             int64
score_team                       int64
score_opponent                   int64
shots_team                     float64
shots_opponent                 float64
saves_team                     float64
saves_opponent                 float64
powerplay_team                 float64
powerplay_opponent             float64
weakening_team                 float64
weakening_opponent             float64
faceoff_team                   float64
faceoff_opponent               float64
result                          object
season                          object
h_a                      

In [16]:
df_teams = pd.concat([df_home, df_away]).reset_index(drop=True)

In [17]:
 df_teams = df_teams.sort_values(['team', 'date']).reset_index(drop=True)

In [18]:
df_teams['win'] = 0
df_teams.loc[df_teams['result'] == 'win', 'win'] = 1
df_teams['draw'] = 0
df_teams.loc[df_teams['result'] == 'draw', 'draw'] = 1
df_teams['lost'] = 0
df_teams.loc[df_teams['result'] == 'lost', 'lost'] = 1   

In [19]:
#groupby - zgrupovanie odpvedajucich si dat, rolling - beriem iba cast tych dat, na ktore beriem priemer, shift - posunutie tych hodnot v ramci tych zgrupovanych skupin
df_teams['win_R5'] = df_teams.groupby(['team', 'season'])['win'].rolling(5, min_periods=1).mean().values
df_teams['win_R5'] =df_teams.groupby(['team', 'season'])['win_R5'].shift(fill_value=0)

In [20]:
df_teams['draw_R5'] = df_teams.groupby(['team', 'season'])['draw'].rolling(5, min_periods=1).mean().values
df_teams['draw_R5'] =df_teams.groupby(['team', 'season'])['draw_R5'].shift(fill_value=0)


In [21]:
df_teams['lost_R5'] = df_teams.groupby(['team', 'season'])['lost'].rolling(5, min_periods=1).mean().values
df_teams['lost_R5'] =df_teams.groupby(['team', 'season'])['lost_R5'].shift(fill_value=0)

In [22]:
df_teams['scored_avg_R5'] = df_teams.groupby(['team', 'season'])['score_team'].rolling(5, min_periods=1).mean().values
df_teams['scored_avg_R5'] =df_teams.groupby(['team', 'season'])['scored_avg_R5'].shift(fill_value=0)

df_teams['conceded_avg_R5'] = df_teams.groupby(['team', 'season'])['score_opponent'].rolling(5, min_periods=1).mean().values
df_teams['conceded_avg_R5'] =df_teams.groupby(['team', 'season'])['conceded_avg_R5'].shift(fill_value=0)

In [23]:
df_teams['matchday'] = df_teams.groupby(['team', 'season']).cumcount()+1
df_teams['matchday_h_a'] = df_teams.groupby(['team', 'season', 'h_a']).cumcount() +1 

In [24]:
df_teams['scored_cum'] = df_teams.groupby(['team', 'season'])['score_team'].cumsum()
df_teams['scored_cum_prev'] =df_teams.groupby(['team', 'season'])['scored_cum'].shift(fill_value=0)
df_teams['scored_cum_prev_avg'] = df_teams['scored_cum_prev'] / (df_teams['matchday']-1)
    

In [25]:
df_teams['conceded_cum'] = df_teams.groupby(['team', 'season'])['score_opponent'].cumsum()
df_teams['conceded_cum_prev'] =df_teams.groupby(['team', 'season'])['conceded_cum'].shift(fill_value=0)
df_teams['conceded_cum_prev_avg'] = df_teams['conceded_cum_prev'] / (df_teams['matchday']-1)

In [26]:
df_teams['H2H_W'] = df_teams.groupby(['team', 'opponent'])['win'].cumsum()
df_teams['H2H_W'] =df_teams.groupby(['team', 'opponent'])['H2H_W'].shift(fill_value=0)

In [27]:
df_teams['H2H_D'] = df_teams.groupby(['team', 'opponent'])['draw'].cumsum()
df_teams['H2H_D'] =df_teams.groupby(['team', 'opponent'])['H2H_D'].shift(fill_value=0)

In [28]:
df_teams['H2H_L'] = df_teams.groupby(['team', 'opponent'])['lost'].cumsum()
df_teams['H2H_L'] =df_teams.groupby(['team', 'opponent'])['H2H_L'].shift(fill_value=0)

In [29]:
df_teams[['H2H_W', 'H2H_D', 'H2H_L']] =df_teams[['H2H_W', 'H2H_D', 'H2H_L']].div(df_teams[['H2H_W', 'H2H_D', 'H2H_L']].sum(axis=1),axis=0).fillna(0)

In [30]:
df_teams

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a,win,draw,lost,win_R5,draw_R5,lost_R5,scored_avg_R5,conceded_avg_R5,matchday,matchday_h_a,scored_cum,scored_cum_prev,scored_cum_prev_avg,conceded_cum,conceded_cum_prev,conceded_cum_prev_avg,H2H_W,H2H_D,H2H_L
0,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,0.00,0.00,0.0,0.000000,0.000000,1,1,4,0,NaN,1,0,NaN,0.000000,0.000000,0.000000
1,6,83436,7,2015-09-13,banská bystrica,košice,STEEL Arena Košice,2741,1,2,0,1,0,0.0500,0.0000,1.0000,0.9500,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,away,1,0,0,1.00,0.00,0.0,4.000000,1.000000,2,1,5,4,4.000000,1,1,1.000000,0.000000,0.000000,0.000000
2,13,83443,14,2015-09-15,banská bystrica,piešťany,ZŠ Banská Bystrica,1973,1,1,1,2,1,0.0513,0.0333,0.9643,0.9512,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,1.00,0.00,0.0,2.500000,0.500000,3,2,7,5,2.500000,2,1,0.500000,0.000000,0.000000,0.000000
3,17,83447,18,2015-09-18,banská bystrica,martin,ZŠ Martin,1700,1,2,1,1,2,0.0278,0.0714,0.9286,0.9722,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2015/2016,away,0,1,0,1.00,0.00,0.0,2.333333,0.666667,4,2,8,7,2.333333,4,2,0.666667,0.000000,0.000000,0.000000
4,22,83452,23,2015-09-20,banská bystrica,poprad,ZŠ Banská Bystrica,2058,1,3,0,1,0,0.0286,0.0000,1.0000,0.9714,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,draw,2015/2016,home,0,1,0,0.75,0.25,0.0,2.000000,1.000000,5,3,9,8,2.000000,4,4,1.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,1057,101759,302,2019-02-15,žilina,újbuda,ZŠ Žilina,581,2,0,1,3,2,0.0732,0.0541,0.9459,0.9268,0.0000,0.3333,0.6667,1.0000,0.4576,0.5424,win,2018/2019,home,1,0,0,0.00,0.40,0.6,2.400000,4.800000,51,26,130,127,2.540000,199,197,3.940000,0.500000,0.250000,0.250000
2788,1067,101769,312,2019-02-17,žilina,nitra,Nitra Aréna,1993,1,0,2,1,3,0.0476,0.0508,0.9655,0.9524,0.1667,0.2500,0.7500,0.8333,0.2273,0.7727,lost,2018/2019,away,0,0,1,0.20,0.40,0.4,2.200000,4.000000,52,26,131,130,2.549020,202,199,3.901961,0.285714,0.047619,0.666667
2789,1068,101770,313,2019-02-22,žilina,liptovský mikuláš,ZŠ Žilina,842,3,0,0,6,1,0.2000,0.0270,0.9730,0.8000,0.0000,0.0000,1.0000,1.0000,0.4800,0.5200,win,2018/2019,home,1,0,0,0.20,0.40,0.4,2.400000,3.600000,53,27,137,131,2.519231,203,202,3.884615,0.466667,0.133333,0.400000
2790,1079,101781,324,2019-02-24,žilina,košice,ZŠ Žilina,815,0,0,3,1,7,0.0333,0.1707,0.8293,0.9667,0.0000,0.2500,0.7500,1.0000,0.5472,0.4528,lost,2018/2019,home,0,0,1,0.40,0.20,0.4,2.800000,2.800000,54,28,138,137,2.584906,210,203,3.830189,0.142857,0.285714,0.571429


In [31]:
df_teams['Num_of_periods'] = df_teams[['win_periods_team','draw_periods','win_periods_opponent']].sum(axis = 1).values
df_teams['points'] = 0

df_teams.loc[ (df_teams['score_team'] > df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 3) ,'points'] = 3 
df_teams.loc[ (df_teams['score_team'] < df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 3) ,'points'] = 0 
df_teams.loc[ (df_teams['score_team'] > df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 4) ,'points'] = 2 
df_teams.loc[ (df_teams['score_team'] < df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 4) ,'points'] = 1 

In [32]:
df_teams

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a,win,draw,lost,win_R5,draw_R5,lost_R5,scored_avg_R5,conceded_avg_R5,matchday,matchday_h_a,scored_cum,scored_cum_prev,scored_cum_prev_avg,conceded_cum,conceded_cum_prev,conceded_cum_prev_avg,H2H_W,H2H_D,H2H_L,Num_of_periods,points
0,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,0.00,0.00,0.0,0.000000,0.000000,1,1,4,0,NaN,1,0,NaN,0.000000,0.000000,0.000000,3,3
1,6,83436,7,2015-09-13,banská bystrica,košice,STEEL Arena Košice,2741,1,2,0,1,0,0.0500,0.0000,1.0000,0.9500,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,away,1,0,0,1.00,0.00,0.0,4.000000,1.000000,2,1,5,4,4.000000,1,1,1.000000,0.000000,0.000000,0.000000,3,3
2,13,83443,14,2015-09-15,banská bystrica,piešťany,ZŠ Banská Bystrica,1973,1,1,1,2,1,0.0513,0.0333,0.9643,0.9512,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,1.00,0.00,0.0,2.500000,0.500000,3,2,7,5,2.500000,2,1,0.500000,0.000000,0.000000,0.000000,3,3
3,17,83447,18,2015-09-18,banská bystrica,martin,ZŠ Martin,1700,1,2,1,1,2,0.0278,0.0714,0.9286,0.9722,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2015/2016,away,0,1,0,1.00,0.00,0.0,2.333333,0.666667,4,2,8,7,2.333333,4,2,0.666667,0.000000,0.000000,0.000000,4,1
4,22,83452,23,2015-09-20,banská bystrica,poprad,ZŠ Banská Bystrica,2058,1,3,0,1,0,0.0286,0.0000,1.0000,0.9714,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,draw,2015/2016,home,0,1,0,0.75,0.25,0.0,2.000000,1.000000,5,3,9,8,2.000000,4,4,1.000000,0.000000,0.000000,0.000000,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,1057,101759,302,2019-02-15,žilina,újbuda,ZŠ Žilina,581,2,0,1,3,2,0.0732,0.0541,0.9459,0.9268,0.0000,0.3333,0.6667,1.0000,0.4576,0.5424,win,2018/2019,home,1,0,0,0.00,0.40,0.6,2.400000,4.800000,51,26,130,127,2.540000,199,197,3.940000,0.500000,0.250000,0.250000,3,3
2788,1067,101769,312,2019-02-17,žilina,nitra,Nitra Aréna,1993,1,0,2,1,3,0.0476,0.0508,0.9655,0.9524,0.1667,0.2500,0.7500,0.8333,0.2273,0.7727,lost,2018/2019,away,0,0,1,0.20,0.40,0.4,2.200000,4.000000,52,26,131,130,2.549020,202,199,3.901961,0.285714,0.047619,0.666667,3,0
2789,1068,101770,313,2019-02-22,žilina,liptovský mikuláš,ZŠ Žilina,842,3,0,0,6,1,0.2000,0.0270,0.9730,0.8000,0.0000,0.0000,1.0000,1.0000,0.4800,0.5200,win,2018/2019,home,1,0,0,0.20,0.40,0.4,2.400000,3.600000,53,27,137,131,2.519231,203,202,3.884615,0.466667,0.133333,0.400000,3,3
2790,1079,101781,324,2019-02-24,žilina,košice,ZŠ Žilina,815,0,0,3,1,7,0.0333,0.1707,0.8293,0.9667,0.0000,0.2500,0.7500,1.0000,0.5472,0.4528,lost,2018/2019,home,0,0,1,0.40,0.20,0.4,2.800000,2.800000,54,28,138,137,2.584906,210,203,3.830189,0.142857,0.285714,0.571429,3,0


In [33]:
df_teams['points_cum'] = df_teams.groupby(['team', 'season'])['points'].cumsum()
df_teams['points_cum_prev'] =df_teams.groupby(['team', 'season'])['points_cum'].shift(fill_value=0)    
df_teams['points_cum_prev_avg'] = df_teams['points_cum_prev'] / (df_teams['matchday']-1)

In [34]:
 
df_teams['points_cum_h_a'] = df_teams.groupby(['team', 'season', 'h_a'])['points'].cumsum()
df_teams['points_cum_h_a_prev'] =df_teams.groupby(['team', 'season', 'h_a'])['points_cum_h_a'].shift(fill_value=0)
df_teams['points_cum_h_a_prev_avg'] = df_teams['points_cum_h_a_prev'] / (df_teams['matchday_h_a']-1)
    


In [35]:
df_teams

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a,win,draw,lost,win_R5,draw_R5,lost_R5,scored_avg_R5,conceded_avg_R5,matchday,matchday_h_a,scored_cum,scored_cum_prev,scored_cum_prev_avg,conceded_cum,conceded_cum_prev,conceded_cum_prev_avg,H2H_W,H2H_D,H2H_L,Num_of_periods,points,points_cum,points_cum_prev,points_cum_prev_avg,points_cum_h_a,points_cum_h_a_prev,points_cum_h_a_prev_avg
0,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,0.00,0.00,0.0,0.000000,0.000000,1,1,4,0,NaN,1,0,NaN,0.000000,0.000000,0.000000,3,3,3,0,NaN,3,0,NaN
1,6,83436,7,2015-09-13,banská bystrica,košice,STEEL Arena Košice,2741,1,2,0,1,0,0.0500,0.0000,1.0000,0.9500,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,away,1,0,0,1.00,0.00,0.0,4.000000,1.000000,2,1,5,4,4.000000,1,1,1.000000,0.000000,0.000000,0.000000,3,3,6,3,3.000000,3,0,NaN
2,13,83443,14,2015-09-15,banská bystrica,piešťany,ZŠ Banská Bystrica,1973,1,1,1,2,1,0.0513,0.0333,0.9643,0.9512,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,1.00,0.00,0.0,2.500000,0.500000,3,2,7,5,2.500000,2,1,0.500000,0.000000,0.000000,0.000000,3,3,9,6,3.000000,6,3,3.000000
3,17,83447,18,2015-09-18,banská bystrica,martin,ZŠ Martin,1700,1,2,1,1,2,0.0278,0.0714,0.9286,0.9722,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2015/2016,away,0,1,0,1.00,0.00,0.0,2.333333,0.666667,4,2,8,7,2.333333,4,2,0.666667,0.000000,0.000000,0.000000,4,1,10,9,3.000000,4,3,3.000000
4,22,83452,23,2015-09-20,banská bystrica,poprad,ZŠ Banská Bystrica,2058,1,3,0,1,0,0.0286,0.0000,1.0000,0.9714,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,draw,2015/2016,home,0,1,0,0.75,0.25,0.0,2.000000,1.000000,5,3,9,8,2.000000,4,4,1.000000,0.000000,0.000000,0.000000,4,2,12,10,2.500000,8,6,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,1057,101759,302,2019-02-15,žilina,újbuda,ZŠ Žilina,581,2,0,1,3,2,0.0732,0.0541,0.9459,0.9268,0.0000,0.3333,0.6667,1.0000,0.4576,0.5424,win,2018/2019,home,1,0,0,0.00,0.40,0.6,2.400000,4.800000,51,26,130,127,2.540000,199,197,3.940000,0.500000,0.250000,0.250000,3,3,43,40,0.800000,34,31,1.240000
2788,1067,101769,312,2019-02-17,žilina,nitra,Nitra Aréna,1993,1,0,2,1,3,0.0476,0.0508,0.9655,0.9524,0.1667,0.2500,0.7500,0.8333,0.2273,0.7727,lost,2018/2019,away,0,0,1,0.20,0.40,0.4,2.200000,4.000000,52,26,131,130,2.549020,202,199,3.901961,0.285714,0.047619,0.666667,3,0,43,43,0.843137,9,9,0.360000
2789,1068,101770,313,2019-02-22,žilina,liptovský mikuláš,ZŠ Žilina,842,3,0,0,6,1,0.2000,0.0270,0.9730,0.8000,0.0000,0.0000,1.0000,1.0000,0.4800,0.5200,win,2018/2019,home,1,0,0,0.20,0.40,0.4,2.400000,3.600000,53,27,137,131,2.519231,203,202,3.884615,0.466667,0.133333,0.400000,3,3,46,43,0.826923,37,34,1.307692
2790,1079,101781,324,2019-02-24,žilina,košice,ZŠ Žilina,815,0,0,3,1,7,0.0333,0.1707,0.8293,0.9667,0.0000,0.2500,0.7500,1.0000,0.5472,0.4528,lost,2018/2019,home,0,0,1,0.40,0.20,0.4,2.800000,2.800000,54,28,138,137,2.584906,210,203,3.830189,0.142857,0.285714,0.571429,3,0,46,46,0.867925,37,37,1.370370
